In [1]:
import string
import pandas as pd
import numpy as np
import ast
from timeit import timeit

import multiprocessing as mp

import nltk
from nltk.corpus import stopwords as sw
from nltk.corpus import wordnet as wn
from nltk.corpus import names 
from nltk import wordpunct_tokenize, ngrams
from nltk import WordNetLemmatizer
from nltk import sent_tokenize, word_tokenize
from nltk import pos_tag

from sklearn.base import BaseEstimator, TransformerMixin

### REQUIRED (RUN ONCE/DOWNLOAD): word tag file for lemmatization
#nltk.download('averaged_perceptron_tagger')
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('names')

## Function to read data

In [6]:
def process_file(filepath, headers):
    
    '''
    Takes a .txt file from the Cornell Movie Diaglogs Corpus 
    and returns a pandas dataframe
    
    '''
    lines = []
    
    to_longform  = ['text', 'line_id', 'genre']
    idx_col, tgt_col = (headers[:-1], headers[-1])
    
    with open(filepath, 'r', encoding = 'iso-8859-1') as f:
        for line in f:
            cols = line.split(' +++$+++ ')
            lines.append(cols)
            
    df = pd.DataFrame(lines, columns = headers)
    
    if tgt_col in to_longform:
        if tgt_col != 'text':
            df[tgt_col] = df[tgt_col].apply(lambda x: ast.literal_eval(x))
            
            # Convert wide format column to its own rows/long form
            df = df.set_index(idx_col)[tgt_col].apply(pd.Series).stack()
            df = df.reset_index()
        
            # Rename columns 
            add_idx_col = [tgt_col+'_idx', tgt_col]
            df.columns = idx_col + add_idx_col
        
        else:
            # Copy full text to another column 
            df['words'] = df[tgt_col]
            idx_col += [tgt_col]
            tgt_col = 'words'
            
            # Tokenize and Lemmatize the lines
            prepos = text_preprocess()
            with mp.Pool() as pool:
                df[tgt_col] = pool.map(prepos.transform, df[tgt_col])
                df[tgt_col] = df[tgt_col].apply(lambda x: ' '.join(x))
            
    return df

In [10]:
# Helper func to read list of pronouns
def read_pronoun(file):
    lines = []
    with open(file, 'r', encoding = 'iso-8859-1') as f:
        for line in f:
            cols = line.replace('\n','').split('\t')
            lines += cols
        return lines 

In [11]:
class text_preprocess(BaseEstimator, TransformerMixin):

    def __init__(self, stopwords=None, punct=None,
                 lower=True, strip=True, with_pro=True):
        self.lower      = lower
        self.strip      = strip
        self.with_pro   = with_pro
        self.pronouns   = read_pronoun('../cornell_movie_dialogs_corpus/pronouns.txt')
        # If pron=True stopwords DOES NOT include pronouns (i.e., pronouns are included in corpus) 
        self.stopwords  = [set(sw.words('english')).difference(self.pronouns) if self.with_pro else set(sw.words('english'))][0]
        self.punct      = punct or set(string.punctuation)
        self.lemmatizer = WordNetLemmatizer()

    def fit(self, sent, y=None):
        return self

    # def inverse_transform(self, sent):
    #    return [" ".join(w) for w in sent]

    def transform(self, sent):
        return list(self.tokenize(sent))
        
    
    def tokenize(self, sent):
        # Break the sentence into part of speech tagged tokens
        # print(pos_tag(wordpunct_tokenize(sent)))
        for token, tag in pos_tag(wordpunct_tokenize(sent)):
            # Apply preprocessing to the token
            token = token.lower() if self.lower else token
            token = token.strip() if self.strip else token
            token = token.strip('_') if self.strip else token
            token = token.strip('*') if self.strip else token
            
            # If stopword, ignore token and continue
            if token in self.stopwords:
                continue

            # If punctuation, ignore token and continue
            if all(char in self.punct for char in token):
                continue

            # Lemmatize the token and yield
            lemma = self.lemmatize(token, tag)
            yield lemma
    
    # Takes the word tag to access the appropriate WordNet dict to lemmatize word token,
    # if lemmatized word cannot be found in the folders, funct will treat it as a noun
    def lemmatize(self, token, tag):
        tag = {
            'N': wn.NOUN,
            'V': wn.VERB,
            'R': wn.ADV,
            'J': wn.ADJ
        }.get(tag[0], wn.NOUN)

        return self.lemmatizer.lemmatize(token, tag)

In [12]:
# Testing the text_preprocess class
test_text = "Harry This ain't ...blonde she babble. his like, boring myself."
a = text_preprocess()
b = a.transform(test_text) # The second elements of the tuple are the tags

In [13]:
b

['harry', 'blonde', 'she', 'babble', 'his', 'like', 'bore', 'myself']

## Read character metadata

In [15]:
headers = ['character_id', 'name', 'movie_id', 'movie_title', 'gender', 'position']

file = '../cornell_movie_dialogs_corpus/movie_characters_metadata.txt'

characters = process_file(file, headers)

In [16]:
characters.head()

,character_id,name,movie_id,movie_title,gender,position
0,u0,BIANCA,m0,10 things i hate about you,f,4\n
1,u1,BRUCE,m0,10 things i hate about you,?,?\n
2,u2,CAMERON,m0,10 things i hate about you,m,3\n
3,u3,CHASTITY,m0,10 things i hate about you,?,?\n
4,u4,JOEY,m0,10 things i hate about you,m,6\n


## Clean character metadata

In [17]:
characters.groupby('gender').count()

,character_id,name,movie_id,movie_title,position
gender,,,,,
?,6020,6020,6020,6020,6020
F,45,45,45,45,45
M,150,150,150,150,150
f,921,921,921,921,921
m,1899,1899,1899,1899,1899


^ Originally missing gender for 6,020 characters

In [ ]:
import nltk
nltk.download('names')

In [18]:
# Classify and reduce unknown ('?') gender using nltk names corpus 
characters.loc[np.logical_and(characters.name.str.title().isin(names.words('male.txt')), characters.gender == '?'), 'gender'] = 'm'
characters.loc[np.logical_and(characters.name.str.title().isin(names.words('female.txt')), characters.gender == '?'), 'gender'] = 'f'

LookupError: 
**********************************************************************
  Resource [93mnames[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('names')
  [0m
  Searched in:
    - 'C:\\Users\\JoanWang/nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - 'C:\\Users\\JoanWang\\Anaconda3\\nltk_data'
    - 'C:\\Users\\JoanWang\\Anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\JoanWang\\AppData\\Roaming\\nltk_data'
**********************************************************************


In [19]:
characters.groupby('gender').count()

,character_id,name,movie_id,movie_title,position
gender,,,,,
?,6020,6020,6020,6020,6020
F,45,45,45,45,45
M,150,150,150,150,150
f,921,921,921,921,921
m,1899,1899,1899,1899,1899


In [20]:
characters.loc[characters.gender == 'F', 'gender'] = "f"
characters.loc[characters.gender == 'M', 'gender'] = "m"

In [21]:
characters.groupby('gender').count()

,character_id,name,movie_id,movie_title,position
gender,,,,,
?,6020,6020,6020,6020,6020
f,966,966,966,966,966
m,2049,2049,2049,2049,2049


## Read conversation metadata

In [22]:
file_convos = '../cornell_movie_dialogs_corpus/movie_conversations.txt'

header_convos = ['char_id_from', 'char_id_to', 'movie_id', 'line_id']

convos = process_file(file_convos, header_convos)

In [23]:
len(convos)

304713

In [ ]:
convos.head()

,char_id_from,char_id_to,movie_id,line_id_idx,line_id
0,u0,u2,m0,0,L194
1,u0,u2,m0,1,L195
2,u0,u2,m0,2,L196
3,u0,u2,m0,3,L197
4,u0,u2,m0,0,L198


## Read lines metadata

In [ ]:
file_lines = '../cornell_movie_dialogs_corpus/movie_lines.txt'

header_lines = ['line_id', 'char_id', 'movie_id', 'char_name','text']

lines = process_file(file_lines, header_lines)

In [ ]:
len(lines)

In [ ]:
lines.head()

## Read title metadata

In [ ]:
file_titles = '../cornell_movie_dialogs_corpus/movie_titles_metadata.txt'

header_titles = ['movie_id', 'movie_title', 'movie_year', 'imdb_rating', 'imdb_vote', 'genre']

titles = process_file(file_titles, header_titles)

In [ ]:
titles.head()

# Joining tables

### Joining character metadata and conversations

In [ ]:
char_cols = ['character_id', 'movie_id', 'gender']
conv_cols = ['char_id_from', 'char_id_to', 'line_id']

convo_gender = pd.merge(characters[char_cols], convos[conv_cols], left_on = 'character_id', right_on = 'char_id_from')
convo_gender.rename(columns={'gender':'gender_from'}, inplace=True)

convo_gender = pd.merge(characters[['character_id', 'gender']], convo_gender, left_on = 'character_id', right_on = 'char_id_to')
convo_gender.rename(columns={'gender':'gender_to'}, inplace=True)
convo_gender.drop(['character_id_x', 'character_id_y'], axis=1, inplace = True)

### Joining with words

In [ ]:
line_cols = ['line_id', 'words']

convo_gender_words = pd.merge(convo_gender, lines[line_cols], on = 'line_id')

### Join with movie metadata

In [ ]:
title_cols = ['movie_id', 'movie_year', 'genre']

#use the first genre only
main_genre = titles[titles.genre_idx == 0]

movies = pd.merge(convo_gender_words, main_genre[title_cols], on = 'movie_id')

In [ ]:
movies.head(10)

### Pickle final table

In [26]:
import pickle

#pickle.dump(movies, open('../data/movies.p', 'wb'))
#df = pickle.load(open("movies.p", 'rb'))

In [27]:
#movies_small = movies[:1700000]

#pickle.dump(movies, open('data/movies.p', 'wb'))

In [ ]:
from sklearn.model_selection import train_test_split

movies_train, movies_test= = train_test_split(movies, test_size=0.33, random_state = 42)
movies_train.shape
movies_test.shape

In [ ]:
pickle.dump(movies_train, open('../data/movies_train.p', 'wb'))
pickle.dump(movies_test, open('../data/movies_test.p', 'wb'))